In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras import losses
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Audio
sound_file = 'beep.wav'
from keras.callbacks import *

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [3]:
# load dataset
dataframe = pd.read_csv("notebooks/datasets/reg_seven.csv", sep=',')

In [4]:
dataframe.head(3)

,difficulty30rsi,difficulty90var,fee_to_reward90rsiUSD,hashrate30var,median_transaction_fee7rocUSD,mining_profitability,price30smaUSD,price3wmaUSD,price7wmaUSD,sentinusd90emaUSD,size90trx,top100cap,transactionvalueUSD,priceUSD
0,97.889,1.168546e+12,49.775,9.266842e+25,36.980,7220.0,56.481,91.616,88.156,68391477,0.334,19.962,2592.0,94.715
1,97.889,1.243755e+12,50.838,9.644142e+25,50.270,7990.0,58.991,100.288,93.861,73007549,0.331,20.024,4400.0,107.749
2,97.889,1.315554e+12,51.407,1.004435e+26,51.448,8852.0,61.879,111.703,101.342,77645623,0.329,19.987,4478.0,120.003


In [5]:
dataframe.shape

(2466, 14)

In [6]:
length=dataframe.shape[1]-1

In [7]:
length

13

In [8]:
# split into input (X) and output (Y) variables
X = dataframe.iloc[:,0:length]
y = dataframe['priceUSD']

In [9]:
X.head(3)

,difficulty30rsi,difficulty90var,fee_to_reward90rsiUSD,hashrate30var,median_transaction_fee7rocUSD,mining_profitability,price30smaUSD,price3wmaUSD,price7wmaUSD,sentinusd90emaUSD,size90trx,top100cap,transactionvalueUSD
0,97.889,1.168546e+12,49.775,9.266842e+25,36.980,7220.0,56.481,91.616,88.156,68391477,0.334,19.962,2592.0
1,97.889,1.243755e+12,50.838,9.644142e+25,50.270,7990.0,58.991,100.288,93.861,73007549,0.331,20.024,4400.0
2,97.889,1.315554e+12,51.407,1.004435e+26,51.448,8852.0,61.879,111.703,101.342,77645623,0.329,19.987,4478.0


In [10]:
y=np.ravel(y)

In [11]:
shape=X.shape[1]

In [12]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

In [13]:
estimators=[]

In [14]:
estimators.append(['mixmax',MinMaxScaler()])

In [15]:
estimators.append(['robust',RobustScaler()])

In [16]:
scale=Pipeline(estimators,verbose=True)

In [17]:
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing mixmax, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing robust, total=   0.0s


Pipeline(memory=None,
         steps=[('mixmax', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ['robust',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)]],
         verbose=True)

In [18]:
X_train=scale.transform(X_train)

In [19]:
X_test=scale.transform(X_test)

In [20]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [21]:
# define neural network model
def sequential_model(initializer='normal', activation='relu', neurons=300, NUM_FEATURES=shape):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation=activation, kernel_initializer=initializer))
    # Compile model
    adam=keras.optimizers.adam(lr=lr_schedule(0), amsgrad=True)
    #sgd=keras.optimizers.SGD(learning_rate=0.08, momentum=0.9, nesterov=False)
    model.compile(loss='logcosh', optimizer=adam, metrics=['mae'])
    return model

In [22]:
mcp_save = ModelCheckpoint('trained_models/ANN_reg_seven_new.hdf5', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100,verbose=1, mode='auto')

In [23]:
regressor=KerasRegressor(build_fn=sequential_model, batch_size=64, epochs=5000,verbose=1, shuffle=True,
                         validation_split=0.1,validation_freq=1,
                        use_multiprocessing=True, callbacks=[mcp_save,earlyStopping])

In [24]:
regressor.fit(X_train,y_train)
Audio(sound_file,autoplay=True)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
Instructions for updating:
Use tf.cast instead.
Train on 1774 samples, validate on 198 samples
Epoch 1/5000
1774/1774 [==============================] - 0s 148us/step - loss: 2960.6959 - mae: 2961.3884 - val_loss: 3223.4174 - val_mae: 3224.1106
Epoch 2/5000
1774/1774 [==============================] - 0s 69us/step - loss: 2666.2859 - mae: 2666.9792 - val_loss: 2171.2965 - val_mae: 2171.9895
Epoch 3/5000
1774/1774 [==============================] - 0s 74us/step - loss: 1277.0038 - mae: 1277.6963 - val_loss: 1076.5405 - val_mae: 1077.2334
Epoch 4/5000
1774/1774 [==============================] - 0s 71us/step - loss: 807.7362 - mae: 808.4284 - val_loss: 595.8225 - val_mae: 596.5156
Epoch 5/5000
1774/1774 [==============================] - 0s 67us/step - loss: 482.6652 - mae: 483.3577 - val_loss: 406.8929 - val_mae: 407.5844
Epoch 6/5000
1774/1774 [==============================] - 0s 71us/step - 

Epoch 53/5000
1774/1774 [==============================] - 0s 74us/step - loss: 110.4838 - mae: 111.1665 - val_loss: 159.9146 - val_mae: 160.6041
Epoch 54/5000
1774/1774 [==============================] - 0s 78us/step - loss: 139.2969 - mae: 139.9831 - val_loss: 162.2067 - val_mae: 162.8877
Epoch 55/5000
1774/1774 [==============================] - 0s 75us/step - loss: 127.8751 - mae: 128.5581 - val_loss: 152.1262 - val_mae: 152.8088
Epoch 56/5000
1774/1774 [==============================] - 0s 67us/step - loss: 117.7452 - mae: 118.4275 - val_loss: 140.7971 - val_mae: 141.4862
Epoch 57/5000
1774/1774 [==============================] - 0s 81us/step - loss: 111.0316 - mae: 111.7156 - val_loss: 148.1755 - val_mae: 148.8603
Epoch 58/5000
1774/1774 [==============================] - 0s 77us/step - loss: 110.2887 - mae: 110.9713 - val_loss: 134.1405 - val_mae: 134.8250
Epoch 59/5000
1774/1774 [==============================] - 0s 74us/step - loss: 120.4095 - mae: 121.0916 - val_loss: 154.930

1774/1774 [==============================] - 0s 77us/step - loss: 131.5501 - mae: 132.2316 - val_loss: 157.8573 - val_mae: 158.5441
Epoch 110/5000
1774/1774 [==============================] - 0s 71us/step - loss: 126.0133 - mae: 126.6930 - val_loss: 150.4759 - val_mae: 151.1622
Epoch 111/5000
1774/1774 [==============================] - 0s 78us/step - loss: 121.6184 - mae: 122.2991 - val_loss: 144.5456 - val_mae: 145.2279
Epoch 112/5000
1774/1774 [==============================] - 0s 71us/step - loss: 117.9291 - mae: 118.6115 - val_loss: 139.0129 - val_mae: 139.6959
Epoch 113/5000
1774/1774 [==============================] - 0s 73us/step - loss: 117.4535 - mae: 118.1340 - val_loss: 133.3347 - val_mae: 134.0191
Epoch 114/5000
1774/1774 [==============================] - 0s 77us/step - loss: 117.8287 - mae: 118.5081 - val_loss: 144.2004 - val_mae: 144.8876
Epoch 115/5000
1774/1774 [==============================] - 0s 68us/step - loss: 119.0037 - mae: 119.6855 - val_loss: 132.9919 - val_

1774/1774 [==============================] - 0s 83us/step - loss: 114.5261 - mae: 115.2080 - val_loss: 166.9976 - val_mae: 167.6881
Epoch 166/5000
1774/1774 [==============================] - 0s 81us/step - loss: 146.3458 - mae: 147.0328 - val_loss: 158.5314 - val_mae: 159.2161
Epoch 167/5000
1774/1774 [==============================] - 0s 72us/step - loss: 134.3165 - mae: 134.9989 - val_loss: 162.6772 - val_mae: 163.3631
Epoch 168/5000
1774/1774 [==============================] - 0s 77us/step - loss: 137.6747 - mae: 138.3597 - val_loss: 150.2432 - val_mae: 150.9242
Epoch 169/5000
1774/1774 [==============================] - 0s 75us/step - loss: 133.2098 - mae: 133.8936 - val_loss: 141.9233 - val_mae: 142.6133
Epoch 170/5000
1774/1774 [==============================] - 0s 66us/step - loss: 121.6229 - mae: 122.3075 - val_loss: 142.6572 - val_mae: 143.3468
Epoch 171/5000
1774/1774 [==============================] - 0s 71us/step - loss: 127.3032 - mae: 127.9848 - val_loss: 154.9697 - val_

Epoch 222/5000
1774/1774 [==============================] - 0s 81us/step - loss: 75.7188 - mae: 76.3950 - val_loss: 94.7121 - val_mae: 95.3953
Epoch 223/5000
1774/1774 [==============================] - 0s 72us/step - loss: 72.7079 - mae: 73.3831 - val_loss: 91.5135 - val_mae: 92.1901
Epoch 224/5000
1774/1774 [==============================] - 0s 79us/step - loss: 71.1863 - mae: 71.8612 - val_loss: 119.5874 - val_mae: 120.2706
Epoch 225/5000
1774/1774 [==============================] - 0s 78us/step - loss: 98.5819 - mae: 99.2631 - val_loss: 124.0571 - val_mae: 124.7422
Epoch 226/5000
1774/1774 [==============================] - 0s 74us/step - loss: 89.0431 - mae: 89.7222 - val_loss: 95.1495 - val_mae: 95.8243
Epoch 227/5000
1774/1774 [==============================] - 0s 71us/step - loss: 74.6435 - mae: 75.3231 - val_loss: 93.1134 - val_mae: 93.7963
Epoch 228/5000
1774/1774 [==============================] - 0s 75us/step - loss: 74.3596 - mae: 75.0373 - val_loss: 92.1435 - val_mae: 92.

...

1774/1774 [==============================] - 0s 63us/step - loss: 41.9243 - mae: 42.5897 - val_loss: 60.8607 - val_mae: 61.5386
Epoch 793/5000
1774/1774 [==============================] - 0s 62us/step - loss: 42.2204 - mae: 42.8840 - val_loss: 59.2041 - val_mae: 59.8740
Epoch 794/5000
1774/1774 [==============================] - 0s 64us/step - loss: 42.7084 - mae: 43.3784 - val_loss: 58.4268 - val_mae: 59.1096
Epoch 795/5000
1774/1774 [==============================] - 0s 65us/step - loss: 45.8731 - mae: 46.5415 - val_loss: 60.6201 - val_mae: 61.2876
Epoch 796/5000
1774/1774 [==============================] - 0s 62us/step - loss: 42.8027 - mae: 43.4649 - val_loss: 58.1158 - val_mae: 58.7872
Epoch 797/5000
1774/1774 [==============================] - 0s 61us/step - loss: 45.5621 - mae: 46.2297 - val_loss: 63.8768 - val_mae: 64.5595
Epoch 798/5000
1774/1774 [==============================] - 0s 60us/step - loss: 52.6810 - mae: 53.3563 - val_loss: 59.0677 - val_mae: 59.7364
Epoch 799/5000

1774/1774 [==============================] - 0s 66us/step - loss: 43.4467 - mae: 44.1139 - val_loss: 63.2464 - val_mae: 63.9162
Epoch 850/5000
1774/1774 [==============================] - 0s 63us/step - loss: 43.3964 - mae: 44.0645 - val_loss: 58.4134 - val_mae: 59.0844
Epoch 851/5000
1774/1774 [==============================] - 0s 64us/step - loss: 48.0007 - mae: 48.6705 - val_loss: 66.3471 - val_mae: 67.0249
Epoch 852/5000
1774/1774 [==============================] - 0s 62us/step - loss: 43.1927 - mae: 43.8539 - val_loss: 58.6381 - val_mae: 59.3104
Epoch 853/5000
1774/1774 [==============================] - 0s 61us/step - loss: 45.3449 - mae: 46.0167 - val_loss: 58.6230 - val_mae: 59.2964
Epoch 854/5000
1774/1774 [==============================] - 0s 62us/step - loss: 43.9318 - mae: 44.6017 - val_loss: 63.8757 - val_mae: 64.5525
Epoch 855/5000
1774/1774 [==============================] - 0s 62us/step - loss: 45.0065 - mae: 45.6753 - val_loss: 61.5748 - val_mae: 62.2471
Epoch 856/5000

1774/1774 [==============================] - 0s 77us/step - loss: 41.0404 - mae: 41.7080 - val_loss: 55.4539 - val_mae: 56.1234
Epoch 907/5000
1774/1774 [==============================] - 0s 63us/step - loss: 39.8260 - mae: 40.4864 - val_loss: 56.2695 - val_mae: 56.9318
Epoch 908/5000
1774/1774 [==============================] - 0s 70us/step - loss: 41.9349 - mae: 42.6062 - val_loss: 61.1149 - val_mae: 61.7960
Epoch 909/5000
1774/1774 [==============================] - 0s 62us/step - loss: 42.5749 - mae: 43.2444 - val_loss: 59.3632 - val_mae: 60.0394
Epoch 910/5000
1774/1774 [==============================] - 0s 65us/step - loss: 45.5449 - mae: 46.2131 - val_loss: 61.9144 - val_mae: 62.5852
Epoch 911/5000
1774/1774 [==============================] - 0s 62us/step - loss: 47.5578 - mae: 48.2262 - val_loss: 61.1485 - val_mae: 61.8315
Epoch 912/5000
1774/1774 [==============================] - 0s 66us/step - loss: 45.2237 - mae: 45.8953 - val_loss: 77.7199 - val_mae: 78.4004
Epoch 913/5000

1774/1774 [==============================] - 0s 69us/step - loss: 41.6759 - mae: 42.3444 - val_loss: 56.0882 - val_mae: 56.7637
Epoch 964/5000
1774/1774 [==============================] - 0s 70us/step - loss: 41.7019 - mae: 42.3700 - val_loss: 68.0609 - val_mae: 68.7453
Epoch 965/5000
1774/1774 [==============================] - 0s 64us/step - loss: 45.1580 - mae: 45.8334 - val_loss: 53.8125 - val_mae: 54.4875
Epoch 966/5000
1774/1774 [==============================] - 0s 62us/step - loss: 41.0505 - mae: 41.7146 - val_loss: 54.9707 - val_mae: 55.6341
Epoch 967/5000
1774/1774 [==============================] - 0s 73us/step - loss: 41.2096 - mae: 41.8792 - val_loss: 57.4414 - val_mae: 58.1163
Epoch 968/5000
1774/1774 [==============================] - 0s 75us/step - loss: 45.1680 - mae: 45.8314 - val_loss: 69.8790 - val_mae: 70.5659
Epoch 969/5000
1774/1774 [==============================] - 0s 75us/step - loss: 43.7574 - mae: 44.4289 - val_loss: 62.5368 - val_mae: 63.2185
Epoch 970/5000

1774/1774 [==============================] - 0s 83us/step - loss: 44.4106 - mae: 45.0811 - val_loss: 54.3567 - val_mae: 55.0202
Epoch 1021/5000
1774/1774 [==============================] - 0s 73us/step - loss: 40.8661 - mae: 41.5298 - val_loss: 57.9734 - val_mae: 58.6403
Epoch 1022/5000
1774/1774 [==============================] - 0s 76us/step - loss: 39.7400 - mae: 40.4037 - val_loss: 56.4488 - val_mae: 57.1262
Epoch 1023/5000
1774/1774 [==============================] - 0s 72us/step - loss: 42.2997 - mae: 42.9677 - val_loss: 64.0499 - val_mae: 64.7299
Epoch 1024/5000
1774/1774 [==============================] - 0s 73us/step - loss: 43.7281 - mae: 44.3951 - val_loss: 58.6598 - val_mae: 59.3375
Epoch 1025/5000
1774/1774 [==============================] - 0s 77us/step - loss: 41.1847 - mae: 41.8524 - val_loss: 53.9211 - val_mae: 54.5869
Epoch 1026/5000
1774/1774 [==============================] - 0s 72us/step - loss: 40.1960 - mae: 40.8617 - val_loss: 56.0004 - val_mae: 56.6766
Epoch 10

In [25]:
y_train_pred=regressor.predict(X_train)

1972/1972 [==============================] - 0s 44us/step


In [26]:
r2_score(y_train, y_train_pred) #training score

0.9993038001594295

In [27]:
y_pred=regressor.predict(X_test)

494/494 [==============================] - 0s 30us/step


In [28]:
pd.DataFrame(zip(y_test,y_pred),columns=['y_test','y_pred'])

,y_test,y_pred
0,6912.000,6851.638672
1,6233.000,6186.158203
2,235.065,233.236984
3,609.126,602.136780
4,453.641,454.957306
5,852.382,815.407593
6,342.284,345.002014
7,344.899,348.735504
8,644.487,636.746582
9,3804.000,3780.987793


In [29]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9993465979473524

In [30]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

50.827623938803725

In [31]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

96.82265374558146

In [32]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

3.3442532123727116

In [33]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,50.8276,96.8227,3.34425,0.999347


In [41]:
regressor.model.save('trained_models/ANN_reg_seven_new.h5')

In [35]:
regressor.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               4200      
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 301       
Total params: 185,101
Trainable params: 185,101
Non-trainable params: 0
_________________________________________________________________


In [36]:
pd.DataFrame(zip(y_test,y_pred),columns=['test','pred']).to_csv('predictions/pred_ANN_reg_seven.csv',index=False)

In [37]:
model=load_model('trained_models/ANN_reg_seven_new.h5',compile=False)

In [38]:
y_pred=model.predict(X_test)

In [39]:
mean_absolute_percentage_error(y_test,np.ravel(y_pred))

3.344252629826457

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               4200      
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 301       
Total params: 185,101
Trainable params: 185,101
Non-trainable params: 0
_________________________________________________________________
